In [262]:
"""
Markov Decision Processes (Chapter 17)
First we define an MDP, and the special case of a GridMDP, in which
states are laid out in a 2-dimensional grid. We also represent a policy
as a dictionary of {state: action} pairs, and a Utility function as a
dictionary of {state: number} pairs. We then define the value_iteration
and policy_iteration algorithms.
"""

import random
from collections import defaultdict

import numpy as np

from utils import vector_add, orientations, turn_right, turn_left


class MDP:
    """A Markov Decision Process, defined by an initial state, transition model,
    and reward function. We also keep track of a gamma value, for use by
    algorithms. The transition model is represented somewhat differently from
    the text. Instead of P(s' | s, a) being a probability number for each
    state/state/action triplet, we instead have T(s, a) return a
    list of (p, s') pairs. We also keep track of the possible states,
    terminal states, and actions for each state. [Page 646]"""

    def __init__(self, init, actlist, terminals, transitions=None, reward=None, states=None, gamma=0.9):
        if not (0 < gamma <= 1):
            raise ValueError("An MDP must have 0 < gamma <= 1")

        # collect states from transitions table if not passed.
        self.states = states or self.get_states_from_transitions(transitions)

        self.init = init

        if isinstance(actlist, list):
            # if actlist is a list, all states have the same actions
            self.actlist = actlist

        elif isinstance(actlist, dict):
            # if actlist is a dict, different actions for each state
            self.actlist = actlist

        self.terminals = terminals
        self.transitions = transitions or {}
        if not self.transitions:
            print("Warning: Transition table is empty.")

        self.gamma = gamma

        self.reward = reward or {s: 0 for s in self.states}

        # self.check_consistency()

    def R(self, state):
        """Return a numeric reward for this state."""

        return self.reward[state]

    def T(self, state, action):
        """Transition model. From a state and an action, return a list
        of (probability, result-state) pairs."""

        if not self.transitions:
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]

    def actions(self, state):
        """Return a list of actions that can be performed in this state. By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""

        if state in self.terminals:
            return [None]
        else:
            return self.actlist

    def get_states_from_transitions(self, transitions):
        if isinstance(transitions, dict):
            s1 = set(transitions.keys())
            s2 = set(tr[1] for actions in transitions.values()
                     for effects in actions.values()
                     for tr in effects)
            return s1.union(s2)
        else:
            print('Could not retrieve states from transitions')
            return None

    def check_consistency(self):

        # check that all states in transitions are valid
        assert set(self.states) == self.get_states_from_transitions(self.transitions)

        # check that init is a valid state
        assert self.init in self.states

        # check reward for each state
        assert set(self.reward.keys()) == set(self.states)

        # check that all terminals are valid states
        assert all(t in self.states for t in self.terminals)

        # check that probability distributions for all actions sum to 1
        for s1, actions in self.transitions.items():
            for a in actions.keys():
                s = 0
                for o in actions[a]:
                    s += o[0]
                assert abs(s - 1) < 0.001

In [263]:
class MDP2(MDP):
    """
    Inherits from MDP. Handles terminal states, and transitions to and from terminal states better.
    """

    def __init__(self, init, actlist, terminals, transitions, reward=None, gamma=0.9):
        MDP.__init__(self, init, actlist, terminals, transitions, reward, gamma=gamma)

    def T(self, state, action):
        if action is None:
            return [(0.0, state)]
        else:
            return self.transitions[state][action]

In [383]:
class GridMDP(MDP):
    """A two-dimensional grid MDP, as in [Figure 17.1]. All you have to do is
    specify the grid as a list of lists of rewards; use None for an obstacle
    (unreachable state). Also, you should specify the terminal states.
    An action is an (x, y) unit vector; e.g. (1, 0) means move east."""

    def __init__(self, grid, terminals, init=(0, 0), gamma=.9):
        grid.reverse()  # because we want row 0 on bottom, not on top
        reward = {}
        states = set()
        self.rows = len(grid)
        self.cols = len(grid[0])
        self.grid = grid
        for x in range(self.cols):
            for y in range(self.rows):
                if grid[y][x]:
                    states.add((x, y))
                    reward[(x, y)] = grid[y][x]
        self.states = states
        actlist = orientations
        transitions = {}
        for s in states:
            transitions[s] = {}
            for a in actlist:
                transitions[s][a] = self.calculate_T(s, a)
        MDP.__init__(self, init, actlist=actlist,
                     terminals=terminals, transitions=transitions,
                     reward=reward, states=states, gamma=gamma)

    def calculate_T(self, state, action):
        if action:
            return [(0.8, self.go(state, action)),
                    (0.1, self.go(state, turn_right(action))),
                    (0.1, self.go(state, turn_left(action)))]
        else:
            return [(0.0, state)]

    def T(self, state, action):
        return self.transitions[state][action] if action else [(0.0, state)]

    def go(self, state, direction):
        """Return the state that results from going in this direction."""

        state1 = vector_add(state, direction)
        return state1 if state1 in self.states else state

    def to_grid(self, mapping):
        """Convert a mapping from (x, y) to v into a [[..., v, ...]] grid."""

        return list(reversed([[mapping.get((x, y), None)
                               for x in range(self.cols)]
                              for y in range(self.rows)]))

    def to_arrows(self, policy):
        chars = {(1, 0): '>', (0, 1): '^', (-1, 0): '<', (0, -1): 'v', None: '.'}
        return self.to_grid({s: chars[a] for (s, a) in policy.items()})


# ______________________________________________________________________________


""" [Figure 17.1]
A 4x3 grid environment that presents the agent with a sequential decision problem.
"""

sequential_decision_environment = GridMDP([[-0.04, -0.04, -0.04, +1],
                                           [-0.04, None, -0.04, -1],
                                           [-0.04, -0.04, -0.04, -0.04]],
                                          terminals=[(3, 2), (3, 1)])


# ______________________________________________________________________________


def value_iteration(mdp, epsilon=0.001):
    """Solving an MDP by value iteration. [Figure 17.4]"""

    U1 = {s: 0 for s in mdp.states}
    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    while True:
        U = U1.copy()
        delta = 0
        for s in mdp.states:
            U1[s] = R(s) + gamma * max(sum(p * U[s1] for (p, s1) in T(s, a))
                                       for a in mdp.actions(s))
            delta = max(delta, abs(U1[s] - U[s]))
        if delta <= epsilon * (1 - gamma) / gamma:
            return U


def best_policy(mdp, U):
    """Given an MDP and a utility function U, determine the best policy,
    as a mapping from state to action. [Equation 17.4]"""

    pi = {}
    for s in mdp.states:
        pi[s] = max(mdp.actions(s), key=lambda a: expected_utility(a, s, U, mdp))
    return pi


def expected_utility(a, s, U, mdp):
    """The expected utility of doing a in state s, according to the MDP and U."""

    return sum(p * U[s1] for (p, s1) in mdp.T(s, a))


# ______________________________________________________________________________


def policy_iteration(mdp):
    """Solve an MDP by policy iteration [Figure 17.7]"""

    U = {s: 0 for s in mdp.states}
    pi = {s: random.choice(mdp.actions(s)) for s in mdp.states}
    while True:
        U = policy_evaluation(pi, U, mdp)
        unchanged = True
        for s in mdp.states:
            a = max(mdp.actions(s), key=lambda a: expected_utility(a, s, U, mdp))
            if a != pi[s]:
                pi[s] = a
                unchanged = False
        if unchanged:
            print("Computed Utilities")
            grid_U = mdp.to_grid(U)
            #print(grid_U)
            for entry in grid_U:
                print(entry)
            print('\nOptimal Policy')   
            print(pi)
            return pi
        
def policy_iteration_2(mdp):
    """Solve an MDP by policy iteration [Figure 17.7]"""

    U = {s: 0 for s in mdp.states}
    pi = {s: random.choice(mdp.actions(s)) for s in mdp.states}
    while True:
        U = policy_evaluation(pi, U, mdp)
        unchanged = True
        for s in mdp.states:
            a = max(mdp.actions(s), key=lambda a: expected_utility(a, s, U, mdp))
            if a != pi[s]:
                pi[s] = a
                unchanged = False
        if unchanged:
            print("Computed Utilities")
            grid_U = mdp.to_grid(U)
            #print(grid_U)
            for entry in grid_U:
                print(entry)
            print('\nOptimal Policy')   
            print(pi)
            return pi, np.array(grid_U)


def policy_evaluation(pi, U, mdp, k=20):
    """Return an updated utility mapping U from each state in the MDP to its
    utility, using an approximation (modified policy iteration)."""

    R, T, gamma = mdp.R, mdp.T, mdp.gamma
    for i in range(k):
        for s in mdp.states:
            U[s] = R(s) + gamma * sum(p * U[s1] for (p, s1) in T(s, pi[s]))
    return U

In [384]:
terminal_states=[(3, 2), (3, 1)]
mdp_env = GridMDP([[-0.04, -0.04, -0.04, +1],
                   [-0.04, None, -0.04, -1],
                   [-0.04, -0.04, -0.04, -0.04]],
                   terminals=terminal_states)

In [385]:
from utils import print_table

print_table(mdp_env.to_arrows(policy_iteration(mdp_env)))

Computed Utilities
[0.5094155954147008, 0.649586359613105, 0.7953622428927029, 1.0]
[0.3985112545104691, None, 0.4864404559151057, -1.0]
[0.2964665410943774, 0.2539605460927299, 0.34478839971672015, 0.12994247010553683]

Optimal Policy
{(0, 1): (0, 1), (1, 2): (1, 0), (2, 1): (0, 1), (0, 0): (0, 1), (3, 1): None, (2, 0): (0, 1), (3, 0): (-1, 0), (0, 2): (1, 0), (2, 2): (1, 0), (1, 0): (1, 0), (3, 2): None}
>   >      >   .
^   None   ^   .
^   >      ^   <


In [386]:
def eval_expected_utility(mdp, policy, state, terminals):
    dirs = {"up": (0,1),
            "right": (1,0),
            "left": (-1,0)}
    avg_util = []
    
    for i in range(100000):
        curr_state = state
        discount = mdp.gamma
        cum_util = mdp.R(curr_state)
        counter = 0
        #print(curr_state)
        while curr_state not in terminals:
            check_dir = policy[curr_state]
            next_move = random.choices(['up', 'left', 'right'], weights=[0.8, 0.1, 0.1])[0]
            #print(next_move)
            counter += 1

            if check_dir == (0,1): # north
                curr_state = mdp.go(curr_state, dirs[next_move])

            elif check_dir == (1,0):  # west
                curr_state = mdp.go(curr_state, turn_right(dirs[next_move]))
                
            elif check_dir == (0,-1):  # south
                curr_state = mdp.go(curr_state, turn_right(turn_right(dirs[next_move])))

            else:   # east
                curr_state = mdp.go(curr_state, turn_left(dirs[next_move]))

            cum_util += (discount**counter) * mdp.R(curr_state)
        avg_util.append(cum_util)
        
    return sum(avg_util)/len(avg_util)

In [387]:
from utils import print_table

print_table(mdp_env.to_arrows(policy_iteration(mdp_env)))

Computed Utilities
[0.5094155954147008, 0.649586359613105, 0.7953622428927029, 1.0]
[0.3985112545104691, None, 0.4864404559151057, -1.0]
[0.29646654109402815, 0.25396054609199764, 0.34478839971636, 0.12994247010515378]

Optimal Policy
{(0, 1): (0, 1), (1, 2): (1, 0), (2, 1): (0, 1), (0, 0): (0, 1), (3, 1): None, (2, 0): (0, 1), (3, 0): (-1, 0), (0, 2): (1, 0), (2, 2): (1, 0), (1, 0): (1, 0), (3, 2): None}
>   >      >   .
^   None   ^   .
^   >      ^   <


In [388]:
%%time
optimal_policy, exp_utils_1 = policy_iteration_2(mdp_env)

Computed Utilities
[0.5094155954147008, 0.649586359613105, 0.7953622428927029, 1.0]
[0.3985112545104691, None, 0.4864404559151057, -1.0]
[0.29646654109402715, 0.2539605460919955, 0.34478839971635894, 0.12994247010515267]

Optimal Policy
{(0, 1): (0, 1), (1, 2): (1, 0), (2, 1): (0, 1), (0, 0): (0, 1), (3, 1): None, (2, 0): (0, 1), (3, 0): (-1, 0), (0, 2): (1, 0), (2, 2): (1, 0), (1, 0): (1, 0), (3, 2): None}
Wall time: 2 ms


In [389]:
%%time
eval_expected_utility(mdp_env, optimal_policy, (2,2), terminal_states)

Wall time: 1.11 s


0.7950932381934493

In [390]:
mdp_env_1_times = [3.78, 3.08, 2.57, 3.02, 3.51, 1.49, 2.66, 1.74, 0.967]
mdp_env_1_utils = [[0.5093, 0.6497, 0.7945, 1.0],
                   [0.3979, None, 0.4840, -1.0],
                   [0.2958, 0.2546, 0.3456, 0.1268]]

In [393]:
mdp_env_1_times = np.array(mdp_env_1_times)
mdp_env_1_utils = np.array(mdp_env_1_utils)

In [329]:
terminal_states_2 =[(4,2), (1,0)]
mdp_env_2 = GridMDP([[-0.04, -0.04, -0.04, -0.04, -1],
                   [-0.04, None, -0.04, -0.04, -0.04],
                   [-0.04, +1, -0.04, None, -0.04]],
                   terminals=terminal_states_2)

In [330]:
from utils import print_table

print_table(mdp_env_2.to_arrows(policy_iteration(mdp_env_2)))

Computed Utilities
[0.5228585047669859, 0.4103147846734511, 0.49989161677304544, 0.4018786173926277, -1.0]
[0.6651012621074259, None, 0.6483252924828785, 0.5087508639030285, 0.25180946775265667]
[0.8130319929556795, 1.0, 0.8113728311246183, None, 0.1723205082497349]

Optimal Policy
{(0, 1): (0, -1), (1, 2): (-1, 0), (4, 0): (0, 1), (2, 1): (0, -1), (0, 0): (1, 0), (3, 1): (-1, 0), (2, 0): (-1, 0), (4, 2): None, (0, 2): (0, -1), (2, 2): (0, -1), (1, 0): None, (3, 2): (-1, 0), (4, 1): (-1, 0)}
v   <      v   <      .
v   None   v   <      <
>   .      <   None   ^


In [331]:
%%time
optimal_policy_2 = policy_iteration(mdp_env_2)

Computed Utilities
[0.5228585047669859, 0.4103147846734511, 0.49989161677304544, 0.4018786173926277, -1.0]
[0.6651012621074259, None, 0.6483252924828785, 0.5087508639030285, 0.251809467752548]
[0.8130319929556795, 1.0, 0.8113728311246183, None, 0.17232050824852732]

Optimal Policy
{(0, 1): (0, -1), (1, 2): (-1, 0), (4, 0): (0, 1), (2, 1): (0, -1), (0, 0): (1, 0), (3, 1): (-1, 0), (2, 0): (-1, 0), (4, 2): None, (0, 2): (0, -1), (2, 2): (0, -1), (1, 0): None, (3, 2): (-1, 0), (4, 1): (-1, 0)}
Wall time: 4.99 ms


In [344]:
%%time
eval_expected_utility(mdp_env_2, optimal_policy_2, (4,2), terminal_states_2)

Wall time: 89.6 ms


-1.0

In [345]:
mdp_env_2_times = [0.929, 0.937, 1.74, 4.05, 1.71, 1.87, 2.93, 3.19, 2.95, 3.58, 3.37, 3.71]
mdp_env_2_utils = [[0.5230, 0.4101, 0.4997, 0.4014, -1.0],
                   [0.6652, None, 0.6478, 0.5085, 0.2523],
                   [0.8132, 1.0, 0.8118, None, 0.1732]]

In [348]:
mdp_1_mean = np.mean(mdp_env_1_times)
mdp_2_mean = np.mean(mdp_env_2_times)
print(mdp_1_mean)
print(mdp_2_mean)

2.535222222222222
2.5805000000000002


In [356]:
np.set_printoptions(precision=4, suppress=True)

In [392]:
mdp_env_2_times = np.array(mdp_env_2_times)

avg_dev_1 = np.sum(np.abs(mdp_env_1_times - np.mean(mdp_env_1_times))) / len(mdp_env_1_times)
print(avg_dev_1)

0.7574814814814813


In [361]:
avg_dev_2 = np.sum(np.abs(mdp_env_2_times - np.mean(mdp_env_2_times))) / len(mdp_env_2_times)
print(avg_dev_2)

0.95275


In [399]:
terminal_states_3 =[(4,7), (1,5), (3,1)]
mdp_env_3 = GridMDP([[-0.04, -0.04, -0.04, -0.04, -1],
                   [-0.04, None, -0.04, -0.04, -0.04],
                   [-0.04, +1, -0.04, None, -0.04],
                   [-0.04, -0.04, -0.04, -0.04, -0.04],
                   [-0.04, -0.04, -0.04, -0.04, -0.04],
                   [-0.04, -0.04, -0.04, -0.04, -0.04],
                   [-0.04, -0.04, -0.04, +1, -0.04],
                   [-0.04, -0.04, -0.04, -0.04, -0.04]],
                   terminals=terminal_states_3)

In [404]:
%%time
policy_iteration(mdp_env_3)

Computed Utilities
[0.5120066558005355, 0.40078633192242147, 0.48870133423964374, 0.39190178116992014, -1.0]
[0.6525767873687076, None, 0.6352213945296291, 0.49739622457870963, 0.2740118945898884]
[0.7987680078365839, 1.0, 0.7962302900137203, None, 0.3397016164787702]
[0.6670677441488915, 0.7990964614794538, 0.6562262722894842, 0.5379537878049192, 0.4424379521008215]
[0.5465125567625505, 0.6337516762968789, 0.5468454880338068, 0.6339447872671986, 0.5475037437629233]
[0.4475700951760174, 0.5468454880338068, 0.6566011198792221, 0.7992407727854066, 0.6682963555141838]
[0.4961690409227429, 0.6339447872671985, 0.7992407727854066, 1.0, 0.8003027971766953]
[0.4383053947168699, 0.5475037437629232, 0.6682963555141838, 0.8003027971766953, 0.6684013908935419]

Optimal Policy
{(4, 0): (0, 1), (3, 4): (-1, 0), (4, 3): (0, -1), (3, 1): None, (3, 7): (-1, 0), (4, 6): (0, -1), (0, 2): (1, 0), (0, 5): (1, 0), (2, 2): (1, 0), (1, 0): (1, 0), (2, 5): (-1, 0), (1, 3): (0, 1), (4, 2): (0, -1), (3, 0): (0, 

{(4, 0): (0, 1),
 (3, 4): (-1, 0),
 (4, 3): (0, -1),
 (3, 1): None,
 (3, 7): (-1, 0),
 (4, 6): (0, -1),
 (0, 2): (1, 0),
 (0, 5): (1, 0),
 (2, 2): (1, 0),
 (1, 0): (1, 0),
 (2, 5): (-1, 0),
 (1, 3): (0, 1),
 (4, 2): (0, -1),
 (3, 0): (0, 1),
 (4, 5): (0, -1),
 (3, 3): (0, -1),
 (3, 6): (-1, 0),
 (0, 1): (1, 0),
 (0, 7): (0, -1),
 (2, 4): (-1, 0),
 (1, 2): (1, 0),
 (0, 4): (0, 1),
 (2, 1): (1, 0),
 (2, 7): (0, -1),
 (1, 5): None,
 (3, 2): (0, -1),
 (4, 1): (-1, 0),
 (4, 7): None,
 (4, 4): (0, -1),
 (0, 0): (1, 0),
 (1, 1): (1, 0),
 (0, 3): (0, 1),
 (2, 0): (1, 0),
 (1, 4): (0, 1),
 (0, 6): (0, -1),
 (2, 3): (0, -1),
 (1, 7): (-1, 0),
 (2, 6): (0, -1)}